# Seq2Seq model for text summarization

In [46]:
import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import spacy
import os
import codecs
from io import open
import itertools
import math

In [47]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda:2" if USE_CUDA else "cpu")
SEED = 23

random.seed(SEED)
torch.manual_seed(SEED)

In [48]:
# choose directory to save trained model data if training is interupted
save_dir = os.path.join("preprocessed_data", "save")
# path to data files
train_path = "processed_data/stopwords200-50_train.csv"
test_path = "processed_data/stopwords200-50_test.csv"
val_path = "processed_data/stopwords200-50_val.csv"
gensim_path = "../lt2212-v19-a4/GoogleNews-vectors-negative300.bin"
pre_trained = False # this is not implemented

Load Preprocesses Data
----------------------




In [49]:
import pandas as pd 

train_data = pd.read_csv(train_path) 
# Preview the first 5 lines of the loaded data 
train_data.head()

,story,highlights
0,maria de villota formula one test driver lost ...,f1 test driver leaves uk hospital and returns ...
1,french police grounds arrest gunman mohammed m...,new witness the mood was silent and somber as ...
2,didier drogba scored goal chelsea beat juventu...,chelsea beat juventus 1 0 to give them advanta...
3,martin scorseses hugo nominations among nine c...,the artist has been the unlikely frontrunner i...
4,lowndes county georgia assistant district atto...,kendrick johnsons body was found in a rolled u...


In [50]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token
UNK_token = 3  # Unknown word token

PAD_str = '<pad>' 
SOS_str = '[START]'
EOS_str = '[END]'
UNK_str = '<unk>'

In [51]:
nlp = spacy.load('en_core_web_md')

class Vocab:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.embeddings = {}        
        self.index2word = {PAD_token: PAD_str, SOS_token: SOS_str, EOS_token: EOS_str, UNK_str: UNK_token}
        self.num_words = 4  # Count SOS, EOS, PAD, UNK        
        
    
    def addEmbedding(self,model,word): 
        vector = get_w2v_vectors(model, word)
        self.embeddings[word] = vector

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True
        keep_words = []
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: PAD_str, SOS_token: SOS_str, EOS_token: EOS_str, UNK_str: UNK_token}
        self.num_words = 4 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [52]:
from gensim.models.keyedvectors import KeyedVectors
import numpy as np

def load_gensim_model(path_to_model):
    model =  KeyedVectors.load_word2vec_format(path_to_model, binary=True)
    return model

def get_w2v_vectors(w2v_model, words):
    w2v_vectors = {}
    for word in words:
        try:
            vec = w2v_model.word_vec(word)
            w2v_vectors[word] = vec
        # this exception will occur when a word does not exist in the vocabulary of this model
        except KeyError:
                vec = np.random.rand(1,300)[0]
    return w2v_vectors


In [53]:
# Using the functions defined above, return a populated vocab object and pairs list for the train data set
def loadPrepareVocab(train_data):
    import itertools as iter 
    print("Start preparing training data ...")      
    vocab = Vocab("CNN")
    t = train_data["story"].tolist()
    r = train_data["highlights"].tolist()
    # get the length of the longest sentence
    max_length = len(max(t, key=len).split(' '))
    # Split every line into pairs
    pairs = list(zip(t,r))
    if pre_trained:
        print("Loading gensim model...")
        gensim = load_gensim_model(gensim_path)
        print("Finished loading gensim model.")
    print("Counting word pairs and adding embedding vectors.")
    for pair in pairs:        
        vocab.addSentence(pair[0])
        vocab.addSentence(pair[1])    
        if pre_trained:
            vocab.addEmbedding(gensim,pair[0])
            vocab.addEmbedding(gensim,pair[1])        
    print("Counted words:", vocab.num_words)
    return vocab, pairs, max_length

vocab, pairs, max_length = loadPrepareVocab(train_data)

Start preparing training data ...
Counting word pairs and adding embedding vectors.
Counted words: 10527


In [54]:
# Add validation data to train
validation_data = pd.read_csv(val_path) 
# Preview the first 5 lines of the loaded data 
validation_data.head()

def addVal(validation_data):
    t = validation_data["story"].tolist()
    r = validation_data["highlights"].tolist()
    for words in list(zip(t,r)):     
        vocab.addSentence(words[0])
        vocab.addSentence(words[1]) 
addVal(validation_data)

In [55]:
# used to prepare validation data set to fit model (not finished)
def prepareData(data):
    print("Start preparing data ...") 
    t = data["story"].tolist()
    r = data["highlights"].tolist()
    pairs = list(zip(t,r))
    return pairs

In [56]:
# look at some examples
for pair in pairs[:2]:
    print("Story")
    print(pair[0])
    print("Highlight")
    print(pair[1])

Story
maria de villota formula one test driver lost sight right eye crash earlier month left hospital returned home spain marussia test driver discharged addenbrookes hospital cambridge uk friday f1 team said 32 year old suffered severe facial neck injuries required two operations car plowed stationary truck duxford airfield july 3 course past two weeks maria made significant progress comfortable familiar surroundings plus support wider family friends undoubtedly provide conducive environment maria commence next phase recovery marussia said statement marussia remained close contact medical team addenbrookes hospital since marias admittance would like express gratitude remarkable care attention received marussia f1 car factor crash marussia staff race drivers timo glock charles pic associated team wish maria well next stage recovery management team continue liaise closely maria family provide assistance possible coming months de villota joined britain based marussia july career included

# Prepare data

In [57]:
def indexesFromSentence(lang, sentence):
    sentence_indices = []
    for word in sentence.split(' '):
        try:
            sentence_indices.append(lang.word2index[word])
        except KeyError:
                sentence_indices.append(UNK_token)
    return sentence_indices + [EOS_token]

def embeddingsFromSentence(lang, sentence):
    sentence_embeddings = []
    for word in sentence.split(' '):
        try:
            sentence_embeddings.append(lang.embeddings[word])
        except KeyError:
                vec = np.random.rand(1,300)[0]
    return sentence_embeddings

def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, vocab):
    indexes_batch = [indexesFromSentence(vocab, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, vocab):
    indexes_batch = [indexesFromSentence(vocab, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(vocab, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, vocab)
    output, mask, max_target_len = outputVar(output_batch, vocab)
    return inp, lengths, output, mask, max_target_len




In [58]:
# Look at examples of tensors and binary padding mask
small_batch_size = 2
batches = batch2TrainData(vocab, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[ 4169,  5847],
        [10498,  5848],
        [ 1577,  4695],
        [ 1942,  2816],
        [  981,  1063],
        [ 5253,  1064],
        [ 7036,  5849],
        [ 9446,   151],
        [ 2736,  1288],
        [10499,  5850],
        [ 7544,  1717],
        [  737,   289],
        [ 1942,  4307],
        [  923,   428],
        [ 2651,  2576],
        [  841,  2118],
        [  445,   741],
        [  634,   142],
        [   16,    31],
        [  594,   297],
        [  221,  5847],
        [ 2048,   904],
        [  391,  3650],
        [ 1611,  5851],
        [ 2232,  4695],
        [ 9856,   148],
        [ 7023,  1337],
        [ 4699,  2331],
        [10500,  5852],
        [10501,  5853],
        [10502,  1033],
        [  288,   669],
        [   78,   434],
        [ 3080,  5854],
        [10503,  4695],
        [10474,  5855],
        [ 2140,   347],
        [  264,  5856],
        [  231,  5847],
        [10504,   920],
        [ 4742,  5857],


Look at some examples. Make sure there are not too many UNK tokens, might have to include more words in vocab.

In [59]:
small_batch_size = 1
batches = batch2TrainData(vocab, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

input_variable.size(), target_variable.size()
sample_source = input_variable.transpose(1,0)[0].data.cpu().numpy()
sample_target = target_variable.transpose(1,0)[0].data.cpu().numpy()

print("source:\n%s \n\ncorresponding tensor:\n%s \n" %(' '.join([vocab.index2word[o] for o in sample_source]), sample_source))
print("target:\n%s \n\ncorresponding tensor:\n%s \n" %(' '.join([vocab.index2word[o] for o in sample_target]), sample_target))

source:
affectionately known home city madrid wise man hortaleza luis aragones left legacy helping spains ascension top world football aragones whose death age 75 announced saturday coached national side european championship title 2008 countrys first success major tournament 40 years breakthrough continued successor vicente del bosque continued tiki taka passing style aragones introduced la roja 2010 world cup 2012 european crown without doubt marked road final successful phase felt great deal appreciation towards del bosque told national teams official website knew health problems never imagined would lead always us luis led website tribute atletico madrid club aragones played decade 1964 74 head coach four occasions recently 2001 03 guided atletico la liga title 1977 spanish cup three occasions luis aragones great player coach great person friend said teams president enrique cerezo first foremost atletico want express behalf whole club condolences family aragones final coaching job 

# Seq2Seq model

## Encoder

In [60]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because the input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

## Attention

In [61]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

## Decoder

In [62]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

Define mask



In [63]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

# Train

Initiating models

In [64]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=max_length):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

Training iterations

One thing to note is that when we save our model, we save a tarball
containing the encoder and decoder state_dicts (parameters), the
optimizers’ state_dicts, the loss, the iteration, etc. Saving the model
in this way will give us the ultimate flexibility with the checkpoint.
After loading a checkpoint, we will be able to use the model parameters
to run inference, or we can continue training right where we left off.

In [65]:
def train_model(model_name, vocab, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(vocab, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training model...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'vocab_dict': vocab.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

Setting up model parameters 
---------

Not managed to include pre_trained embeddings, when embedding_size == hidden_size then it is easier to deal with multi-layer architecture because there is no need to treat  the first layers differently (accepting embeddings) from the successive layers (accepting hidden states)

In [66]:
# name the model is saved as
model_name = 'CNN_model'
# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
# Choose if using pre-trained Glove embeddings or not
pre_trained = False # this is not implemented
# Configure model paramters
attn_model = 'dot'
hidden_size = 500 
encoder_n_layers = 4
decoder_n_layers = 4
dropout = 0.1
batch_size = 64

In [67]:
checkpoint_iter = 4000

#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    vocab.__dict__ = checkpoint['vocab_dict']


print('Building encoder and decoder ...')
embedding = nn.Embedding(vocab.num_words, hidden_size)

if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, vocab.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Finished building models.')

Building encoder and decoder ...
Finished building models.


# Training

In [68]:
# Configure training/optimization paramteters
clip = 50.0
teacher_forcing_ratio = 3.0
learning_rate = 0.001
decoder_learning_ratio = 5.0
n_iteration = 20000
print_every = 100
save_every = 500

In [69]:
# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
            
for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
print('Finished building optimizers.')

Building optimizers ...
Finished building optimizers.


In [70]:
# Run training iterations
print("Starting Training...")
train_model(model_name, vocab, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, 'CNN', loadFilename)

Starting Training...
Initializing ...
Training model...
Iteration: 100; Percent complete: 0.5%; Average loss: 7.5890
Iteration: 200; Percent complete: 1.0%; Average loss: 6.6001
Iteration: 300; Percent complete: 1.5%; Average loss: 5.6520
Iteration: 400; Percent complete: 2.0%; Average loss: 4.6346
Iteration: 500; Percent complete: 2.5%; Average loss: 3.9610
Iteration: 600; Percent complete: 3.0%; Average loss: 3.5845
Iteration: 700; Percent complete: 3.5%; Average loss: 3.1525
Iteration: 800; Percent complete: 4.0%; Average loss: 2.7193
Iteration: 900; Percent complete: 4.5%; Average loss: 2.1774
Iteration: 1000; Percent complete: 5.0%; Average loss: 1.7401
Iteration: 1100; Percent complete: 5.5%; Average loss: 1.2684
Iteration: 1200; Percent complete: 6.0%; Average loss: 1.1471
Iteration: 1300; Percent complete: 6.5%; Average loss: 1.1198
Iteration: 1400; Percent complete: 7.0%; Average loss: 1.0896
Iteration: 1500; Percent complete: 7.5%; Average loss: 1.1063
Iteration: 1600; Percen

Iteration: 13100; Percent complete: 65.5%; Average loss: 0.9767
Iteration: 13200; Percent complete: 66.0%; Average loss: 0.9796
Iteration: 13300; Percent complete: 66.5%; Average loss: 0.9696
Iteration: 13400; Percent complete: 67.0%; Average loss: 0.9658
Iteration: 13500; Percent complete: 67.5%; Average loss: 0.9604
Iteration: 13600; Percent complete: 68.0%; Average loss: 0.9645
Iteration: 13700; Percent complete: 68.5%; Average loss: 0.9637
Iteration: 13800; Percent complete: 69.0%; Average loss: 0.9766
Iteration: 13900; Percent complete: 69.5%; Average loss: 0.9803
Iteration: 14000; Percent complete: 70.0%; Average loss: 0.9639
Iteration: 14100; Percent complete: 70.5%; Average loss: 0.9564
Iteration: 14200; Percent complete: 71.0%; Average loss: 0.9584
Iteration: 14300; Percent complete: 71.5%; Average loss: 0.9612
Iteration: 14400; Percent complete: 72.0%; Average loss: 0.9683
Iteration: 14500; Percent complete: 72.5%; Average loss: 0.9702
Iteration: 14600; Percent complete: 73.0

## Testing

In [71]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [72]:
def evaluate(encoder, decoder, searcher, vocab, sentence, max_length=max_length):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(vocab, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [vocab.index2word[token.item()] for token in tokens]
    return decoded_words

In [73]:
def evaluateInput(input_sentence, encoder, decoder, searcher, vocab):
    # Evaluate sentence
    output_words = evaluate(encoder, decoder, searcher, vocab, input_sentence)
    # Format and print response sentence
    output_words[:] = [x for x in output_words if not (x == EOS_str or x == PAD_str)]
    return ' '.join(output_words)

# Evaluation

Load the test file, print the original news article and the highlight togeather with the prediction.

In [74]:
import pandas as pd 

test_data = pd.read_csv(test_path) 
# Preview the first 5 lines of the loaded data 
test_data.head()

,story,highlights
0,77 nato service members wounded saturday attac...,new all of the injured troops are americans an...
1,sudan arrested foreigners disputed region hegl...,those arrested are british norweigan south afr...
2,washington president obama monday gathered tog...,new president obama says we have got to earn t...
3,secretary state condoleezza rice met friday li...,condoleezza rice meets with moammar gadhafi in...
4,deep south plunged deep freeze week time ice s...,new this town is shutting down tomorrow new or...


In [75]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

In [76]:
story, summary = test_data["story"].tolist(), test_data["highlights"].tolist()

In [78]:
for st,su in zip(story,summary):
    pr = evaluateInput(st,encoder, decoder, searcher, vocab)
    
    print("Story: ", st)
    print("Summary: ", su)
    print("---------------------------------------------------------------")
    print("Prediction: ", pr)
    print("---------------------------------------------------------------")

Story:  77 nato service members wounded saturday attack coalition base afghanistan us troops spokeswoman international security assistance force said sunday two afghan civilians killed 25 others also wounded attack occurred eve 10th anniversary al qaedas attack united states 9 11 us army sgt lindsey kibler said none injuries life threatening isaf said wounded expected return duties shortly truck bombing took place central east province wardak killed afghan laborers said shahidullah shahid wardak governors spokesman attack high profile attack pretty significant suicide vehicle bomb gen john r allen commander coalition us forces afghanistan told cnns suzanne malveaux sunday taliban claimed responsibility assault natos international security assistance force confirmed attack carried taliban suicide bomber allen said attack indicates much taliban unable able ejected population many places around country ability influence battlefield many occasions simply go high profile attack view particu

Story:  deep south plunged deep freeze week time ice snow expected fall way gulf mexico forecasters issued winter storm warning orleans suburbs monday afternoon warning city could see half inch snow ice tuesday evening parishes north could see three inches temperatures plunging single digits wednesday national weather service reported wave arctic air settled midwest plains expected spread southeast late monday bringing chance snow several major southern cities tuesday orleans measurable snow fallen 17 times records date back 1880s french quarter restaurants made plans shut tourists worried stranded citys airport town shutting tomorrow cab driver august delaney told cnn bridges going shut schools closed going put kids school buses mayor mitch landrieu declared state emergency warned residents remember happened temperatures dipped toward freezing mark less week ago bridges frozen might remember landrieu said accidents bridges fairly large pile green bridge sometime long ago similar event

Story:  given magnitude japans recession perhaps come little surprise fantasy obsessed animation industry received hard dose reality nobuki mitani animator one lowest paid positions animation yasuo yamaguchi executive director association japanese animators said industry rocked countrys deepest recession since world war ii spread free internet downloading deadly effect said japanese animation roughly 2 billion year industry revenues peaked 2006 since fallen lower advertising revenues lead fewer programs yamaguchi said animation industry important japans economy government helping tough times subsidies watch cnns morgan neill report anime » unlike big screen animated features united states rely almost completely computer animation japan almost features drawn hand labor intensive craft practiced thousands young artists year last six years nobuki mitani working animator filling cells key animations one lowest paid positions animation hierarchy many entry level jobs outsourced philippines 

Story:  march 14 favorite day nerd across country math geeks museums schools private groups elsewhere gather celebrate number pi approximately 314 march 14 3 14 pi day albert einstein born day quick refresher pi defined distance around perfect circle circumference divided distance across diameter also involved calculating area circle volume sphere many mathematical formulas might need sciences throughout history people captivated number way calculate exactly simple division calculator digits go infinitely without pattern numbers 31415926535897932 etc even many digits people would need everyday use folks inspired memorize thousands digits pi even use digits create poetry music pi day one number reeks mystery math may scary pi evidenced widespread revelry pi day one might even say gasp cool like pi days even house representatives supported designation march 14 national pi day 2009 countries day written month friday 14 3 looks less like pi pi day acquired taste mathematician jonathan borw

Story:  massachusetts woman found guilty first degree murder pregnant friend sentenced life without possibility parole according worcester superior court julie corey 35 time crime convicted 2009 found bludgeoned strangled friend darlene haynes 23 cutting baby victims womb haynes body found apartment july 27 2009 eight months pregnant time death days murder haynes baby found corey hampshire dna evidence fingerprint bottle crime scene tied corey killing district attorneys office said according trial testimony corey tried pass baby family friends statement said probably horrific case office seen said worcester county district attorney joseph early jr woman killed baby surprise sentence mandatory sentence massachusetts statutory law notice appeal filed said coreys attorney michael c wilcox
Summary:  julie corey was convicted in 2009 of murdering her pregnant friend darlene haynes haynes baby was found with corey days after the murder court says corey tried to pass the baby off as her own c

...

Story:  house democrats bracing rough election night next tuesday top leaders making major push rank file members open wallets help save vulnerable democrats danger losing seats house democratic leader nancy pelosi rep steve israel york chairman democratic congressional campaign committee held conference call tuesday afternoon house democrats told time pony gave friday deadline according source call dozen house members two democratic candidates pledged almost 500000 session several democratic sources told cnn members expecting republicans pick seats still believe many house democrats competitive districts positioned survive get support dccc invested heavily field operations 950 staff 40 districts surge outside money gop groups television radio spots combined contributions several wealthy gop candidates democrats concerned memo circulated house democrats dccc chairman argued uptick gop spending outside groups something committee prepared counter israel wrote despite effort republicans a

Story:  researcher warned hurricane katrinas catastrophic damage sued louisiana state university wednesday accusing university officials pushing job criticism army corps engineers ivor van heerden led investigation failure levee system surrounding orleans flooded levees failed 2005 storm inquiry put much blame disaster corps lawsuit called important funding source lsu suit accuses lsu putting bureaucratic interests university officials health safety millions people live path hurricanes threaten gulf coast every year argues school officials engaged illegal reprisal van heerden deputy director lsus hurricane center critical research testimony levee system van heerden told 2009 contract expires may would renewed immediate response lsu officials complaint filed state court baton rouge wednesday statement issued wednesday afternoon lsu chancellor michael martin said decision renew van heerdens job made martin joined university administration said reviewed issue confident process handled app

Story:  protests gaining steam baltimore man died devastating injury allegedly suffered police custody demonstrators vowed keep taking streets get justice start protesters say looking answers happened freddie gray seems like questions controversial case keep growing protesters rallied baltimore city hall thursday another march planned saturday heres look key issues protesters asking police say gray saw police april 12 started running within minutes caught arrested finding knife pocket protesters echoed claims gray familys attorney argues police probable cause pursue chased running black baltimore police union attorney michael davey told reporters wednesday officers every right chase gray supreme court case states high crime area flee police unprovoked police legal ability pursue said type incident need probable cause arrest need reasonable suspicion make stop autopsy says gray died severe spinal cord injury family says voice box crushed neck snapped slipped coma later died police say s

Story:  peaches geldof daughter musician bob geldof late paula yates died heroin overdose relapsing use illegal drug following attempts give inquest heard wednesday death 25 year old tv host april home wrotham kent southeast london shocked entertainment world detective chief inspector paul fotheringham kent police told inquest geldof addicted heroin number years ceased taking drug recently geldof two children younger 2 supported drug treatment workers 2½ years prescribed methadone said said wanted come methadone completely following reduction plan drug test november indicated taking illegal drugs said however february witnesses reported suspicions started using heroin high grade heroin fotheringhams report inquest gave details geldofs death police searched house died found black cloth bag hidden cupboard door spare bedroom said inside bag containing tests proved heroin well bag citric acid used make heroin soluble water injection according report bag also held dozens syringes sealed ot

Story:  head libyas opposition government told reporters saturday welcomed call friday russian president dmitry medvedev moammar gadhafi step medvedevs statement echoing stance american european leaders appeared indicate closing diplomatic window longtime libyan strongman chairman national transitional council mustafa abdul jalil said offered amnesty gadhafi loyalists defect demise regime reiterated negotiation solution gadhafis departure happens elections constitutional referendum held within year jailil said opposition stronghold benghazi interview cnn jalil said council sold shipment oil china 160 million confirmation sale expected buttress political economic credibility fledgling rebel power leader said movement financial crisis seeking loans outside country council happy political support ask russia arms economic aid jalil said unrest libya persisted months opposition members demand end gadhafis nearly 42 year rule jalil marked 100th day nations civil war see victories emerging na